In [ ]:


apps_dates = [['id', 'subject_identifier', 'visit_code']]
for ap in Appointment.objects.all():
    apps_dates.append([ap.id, ap.registered_subject.subject_identifier, ap.visit_definition.code])

import pandas as pd
df = pd.DataFrame(apps_dates)
df.to_csv('existing_appointment.csv', encoding='utf-8')


#Appointment dates
import pytz
import uuid
from dateutil import parser
TIME_ZONE = 'Africa/Gaborone'
tz = pytz.timezone(TIME_ZONE)
from cancer_subject.models import Appointment
from django.core.exceptions import ValidationError
fname = '/home/django/source/appointment_dates.csv'
f = open(fname, 'r')
lines = f.readlines()
lines.pop(0)
headers = lines[0]
headers = headers.strip()
headers = headers.split(',')
del headers[0]
lines.pop(0)
app_data = []
missing = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split(',')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(headers)
        print('---------------------------------')
        print(line)
        print(len(line))
        print(len(headers))
        raise ValidationError('Line is not the right size')
    data.update(id=uuid.UUID(data.get('id')))
    if data['created']:
        data.update(created=parser.parse(data.get('created')).replace(tzinfo=tz))
    else:
        data.update(created=None)
    if data['best_appt_datetime']:
        data.update(best_appt_datetime=parser.parse(data.get('best_appt_datetime')).replace(tzinfo=tz))
    else:
        data.update(best_appt_datetime=None)
    if data['modified']:
        data.update(modified=parser.parse(data.get('modified')).replace(tzinfo=tz))
    else:
        data.update(modified=None)
    if data['appt_datetime']:
        data.update(appt_datetime=parser.parse(data.get('appt_datetime')).replace(tzinfo=tz))
    else:
        data.update(appt_datetime=None)
    if data['appt_close_datetime']:
        data.update(appt_close_datetime=parser.parse(data.get('appt_close_datetime')).replace(tzinfo=tz))
    else:
        data.update(appt_close_datetime=None)
    app_data.append(data)
    count += 0
    try:
        appointment = Appointment.objects.get(id=data.get('id'))
    except Appointment.DoesNotExist:
        missing.append(data.get('id'))
        print('Missing appointment', data.get('id'), data.get('subject_identifier'))
    else:
        appointment.created = data.get('created')
        appointment.best_appt_datetime = data.get('best_appt_datetime')
        appointment.modified = data.get('modified')
        appointment.appt_datetime = data.get('appt_datetime')
        appointment.appt_close_datetime = data.get('appt_close_datetime')
        appointment.appt_status = data.get('appt_status')
        appointment.save_base(raw=True)